# CasualMarket MCP 工具測試手冊

## 🛠️ 問題診斷與解決

### 遇到的問題
- **錯誤**: `McpError: Connection closed`
- **原因**: MCP 伺服器參數配置錯誤

### 解決方案
更正 `params` 配置：

```python
# ❌ 錯誤的配置
params = {
    "command": "uv",
    "args": [
        "--directory",
        "/Users/sacahan/Documents/workspace/CasualMarket",
        "run",
        "src.main:main",  # 這裡是錯誤的
    ],
}

# ✅ 正確的配置  
params = {
    "command": "uv",
    "args": [
        "run", 
        "--directory", "/Users/sacahan/Documents/workspace/CasualMarket",
        "python", "-m", "src.main"  # 這裡是正確的
    ],
}
```

### 正確的工具調用方式
使用 `server.session.call_tool()` 而不是直接調用方法：

```python
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_taiwan_stock_price",
        {"symbol": "2330"}
    )
```

## 🎨 美化 JSON 結果顯示

### 新增功能
本測試手冊現在包含美化的 JSON 結果顯示功能：

#### 1. 美化股票查詢結果
- **表格式顯示**: 將股票資訊以美觀的表格形式展示
- **彩色標記**: 漲跌用不同顏色標示（紅色下跌、綠色上漲）
- **格式化數據**: 自動格式化價格、百分比、千分位等

#### 2. 通用 JSON 美化工具
```python
# 美化顯示任意 JSON 資料
pretty_print_json(data, title="我的 JSON 資料")

# 美化顯示 MCP 工具結果  
display_mcp_result(result, tool_name="股票查詢")

# 通用 MCP 工具測試（含美化顯示）
await test_any_mcp_tool_with_beauty(
    "get_taiwan_stock_price", 
    {"symbol": "2330"}, 
    "查詢台積電股價"
)
```

#### 3. 特色功能
- **摺疊式 JSON**: 點擊展開/摺疊詳細 JSON 資料
- **成功/失敗狀態**: 清楚的視覺回饋
- **時間戳記**: 顯示工具執行時間
- **錯誤處理**: 優雅的錯誤顯示

---

In [1]:
from agents.mcp import MCPServerStdio
import os
from IPython.display import Markdown, display
import json

# 修復後的參數設定 - 這是正確的配置
params = {
    "command": "uv",
    "args": [
        "run", 
        "--directory", "/Users/sacahan/Documents/workspace/CasualMarket",
        "python", "-m", "src.main"
    ],
}

## 1. 交易相關工具測試

### 1.1 取得台灣股票即時價格

In [7]:
# 測試不同的股票代碼（MCP 連接已修復）- 美化 JSON 結果顯示
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_taiwan_stock_price",
        {"symbol": "2317"}
    )
    print(json.dumps(result.structuredContent, ensure_ascii=False, indent=2))


{
  "success": true,
  "data": {
    "symbol": "2317",
    "company_name": "鴻海精密工業股份有限公司",
    "current_price": 226.5,
    "change": 3.5,
    "change_percent": 0.01569506726457399,
    "volume": 116723,
    "high": 230.0,
    "low": 222.5,
    "open": 224.0,
    "previous_close": 223.0,
    "last_update": "2025-10-17T14:30:00"
  },
  "error": null,
  "tool": "stock_price",
  "timestamp": "2025-10-19T22:55:52.785473",
  "metadata": {
    "symbol": "2317"
  }
}


In [8]:
# 測試：使用公司名稱查詢
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_taiwan_stock_price",
        {"symbol": "台積電"}
    )
    print(json.dumps(result.structuredContent, ensure_ascii=False, indent=2))

{
  "success": true,
  "data": {
    "symbol": "2330",
    "company_name": "台灣積體電路製造股份有限公司",
    "current_price": 1450.0,
    "change": -35.0,
    "change_percent": -0.02356902356902357,
    "volume": 37785,
    "high": 1465.0,
    "low": 1450.0,
    "open": 1455.0,
    "previous_close": 1485.0,
    "last_update": "2025-10-17T14:30:00"
  },
  "error": null,
  "tool": "stock_price",
  "timestamp": "2025-10-19T22:56:30.325087",
  "metadata": {
    "symbol": "台積電"
  }
}


In [9]:
# 測試：查詢ETF
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_taiwan_stock_price",
        {"symbol": "取得0050股價"}
    )
    print(json.dumps(result.structuredContent, ensure_ascii=False, indent=2))

{
  "success": false,
  "data": null,
  "error": "無效的股票代號格式: 取得0050股價",
  "tool": "stock_price",
  "timestamp": "2025-10-19T22:57:03.654447",
  "metadata": {
    "symbol": "取得0050股價"
  }
}


### 1.2 模擬股票買賣

In [11]:
# 測試：市價買入
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "buy_taiwan_stock",
        {
            "symbol": "0050",
            "quantity": 5000
        }
    )
    print(json.dumps(result.structuredContent, ensure_ascii=False, indent=2))

{
  "success": true,
  "data": {
    "order": {
      "action": "buy",
      "symbol": "0050",
      "name": "元大台灣卓越50證券投資信託基金",
      "quantity": 5000,
      "price": 62.05,
      "total_amount": 310250.0,
      "current_price": 62.05,
      "order_type": "market",
      "executed": true,
      "message": "市價單立即成交",
      "timestamp": "2025-10-17T14:30:00"
    }
  },
  "error": null,
  "tool": "stock_trading",
  "timestamp": "2025-10-19T22:59:17.767048",
  "metadata": {
    "status": "success"
  }
}


In [12]:
# 測試：限價買入
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "buy_taiwan_stock",
        {
            "symbol": "2330",
            "quantity": 2000,
            "price": 500
        }
    )
    print(json.dumps(result.structuredContent, ensure_ascii=False, indent=2))

{
  "success": true,
  "data": {
    "order": {
      "action": "buy",
      "symbol": "2330",
      "name": "台灣積體電路製造股份有限公司",
      "quantity": 2000,
      "price": 500.0,
      "total_amount": 1000000.0,
      "current_price": 1450.0,
      "order_type": "limit",
      "executed": false,
      "message": "限價買單無法成交（出價 500.0 < 市價 1450.0）",
      "timestamp": "2025-10-17T14:30:00"
    }
  },
  "error": null,
  "tool": "stock_trading",
  "timestamp": "2025-10-19T23:02:43.644977",
  "metadata": {
    "status": "failed"
  }
}


In [13]:
# 測試：市價賣出
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "sell_taiwan_stock",
        {
            "symbol": "2330",
            "quantity": 1000
        }
    )
    print(json.dumps(result.structuredContent, ensure_ascii=False, indent=2))

{
  "success": true,
  "data": {
    "order": {
      "action": "sell",
      "symbol": "2330",
      "name": "台灣積體電路製造股份有限公司",
      "quantity": 1000,
      "price": 1450.0,
      "total_amount": 1450000.0,
      "current_price": 1450.0,
      "order_type": "market",
      "executed": true,
      "message": "市價單立即成交",
      "timestamp": "2025-10-17T14:30:00"
    }
  },
  "error": null,
  "tool": "stock_trading",
  "timestamp": "2025-10-19T23:03:33.519428",
  "metadata": {
    "status": "success"
  }
}


In [14]:
# 測試：限價賣出
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "sell_taiwan_stock",
        {
            "symbol": "2330",
            "quantity": 1000,
            "price":550
        }
    )
    print(json.dumps(result.structuredContent, ensure_ascii=False, indent=2))

{
  "success": true,
  "data": {
    "order": {
      "action": "sell",
      "symbol": "2330",
      "name": "台灣積體電路製造股份有限公司",
      "quantity": 1000,
      "price": 550.0,
      "total_amount": 550000.0,
      "current_price": 1450.0,
      "order_type": "limit",
      "executed": true,
      "message": "限價賣單成交（售價 550.0 <= 市價 1450.0）",
      "timestamp": "2025-10-17T14:30:00"
    }
  },
  "error": null,
  "tool": "stock_trading",
  "timestamp": "2025-10-19T23:04:13.773721",
  "metadata": {
    "status": "success"
  }
}


### 1.3 日交易統計

In [15]:
# 測試：取得日交易統計
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_stock_daily_trading",
        {
            "symbol": "2330",
        }
    )
    print(json.dumps(result.structuredContent, ensure_ascii=False, indent=2))

{
  "success": true,
  "data": {
    "trading_date": "2025-10-17",
    "open_price": 1455.0,
    "high_price": 1465.0,
    "low_price": 1450.0,
    "close_price": 1450.0,
    "volume": 37785,
    "change": -35.0,
    "change_percent": -0.02356902356902357,
    "previous_close": 1485.0
  },
  "error": null,
  "tool": "daily_trading",
  "timestamp": "2025-10-19T23:05:05.834627",
  "metadata": {
    "symbol": "2330"
  }
}


## 2. 財務分析工具測試

### 2.1 損益表與資產負債表

In [16]:
# 測試：取得綜合損益表 (一般業)
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_company_income_statement",
        {"symbol": "2330"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

{
  "success": true,
  "data": {
    "raw_data": {
      "出表日期": "1141019",
      "年度": "114",
      "季別": "2",
      "公司代號": "2330",
      "公司名稱": "台積電",
      "營業收入": "1773045533.00",
      "營業成本": "732281219.00",
      "原始認列生物資產及農產品之利益（損失）": "",
      "生物資產當期公允價值減出售成本之變動利益（損失）": "",
      "營業毛利（毛損）": "1040764314.00",
      "未實現銷貨（損）益": "",
      "已實現銷貨（損）益": "",
      "營業毛利（毛損）淨額": "1040764314.00",
      "營業費用": "169694395.00",
      "其他收益及費損淨額": "-565473.00",
      "營業利益（損失）": "870504446.00",
      "營業外收入及支出": "53426170.00",
      "稅前淨利（淨損）": "923930616.00",
      "所得稅費用（利益）": "165704531.00",
      "繼續營業單位本期淨利（淨損）": "758226085.00",
      "停業單位損益": "",
      "合併前非屬共同控制股權損益": "",
      "本期淨利（淨損）": "758226085.00",
      "其他綜合損益（淨額）": "-222604953.00",
      "合併前非屬共同控制股權綜合損益淨額": "",
      "本期綜合損益總額": "535621132.00",
      "淨利（淨損）歸屬於母公司業主": "759837230.00",
      "淨利（淨損）歸屬於共同控制下前手權益": "",
      "淨利（淨損）歸屬於非控制權益": "-1611145.00",
      "綜合損益總額歸屬於母公司業主": "538015970.00",
      "綜合損益總額歸屬於共同控制下前

In [17]:
# 測試：取得綜合損益表 (金融業)
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_company_income_statement",
        {"symbol": "2884"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

{
  "success": false,
  "data": null,
  "error": "找不到 2884 的損益表資料",
  "tool": "financial_statements",
  "timestamp": "2025-10-20T00:03:05.749787",
  "metadata": {
    "company_code": "2884",
    "statement_type": "綜合損益表"
  }
}


In [18]:
# 測試：取得資產負債表
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_company_balance_sheet",
        {"symbol": "2330"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

{
  "success": true,
  "data": {
    "raw_data": {
      "出表日期": "1141019",
      "年度": "114",
      "季別": "2",
      "公司代號": "2330",
      "公司名稱": "台積電",
      "流動資產": "3264917475.00",
      "非流動資產": "3741432074.00",
      "資產總額": "7006349549.00",
      "流動負債": "1377314334.00",
      "非流動負債": "1012403365.00",
      "負債總額": "2389717699.00",
      "股本": "259326155.00",
      "權益─具證券性質之虛擬通貨": "",
      "資本公積": "73326265.00",
      "保留盈餘": "4430887293.00",
      "其他權益": "-182465738.00",
      "庫藏股票": "0.00",
      "歸屬於母公司業主之權益合計": "4581073975.00",
      "共同控制下前手權益": "",
      "合併前非屬共同控制股權": "",
      "非控制權益": "35557875.00",
      "權益總額": "4616631850.00",
      "預收股款（權益項下）之約當發行股數": "",
      "待註銷股本股數": "0.00",
      "母公司暨子公司所持有之母公司庫藏股股數": "0.00",
      "每股參考淨值": "176.65"
    },
    "key_metrics": {
      "total_assets": "7006349549.00",
      "total_liabilities": "2389717699.00",
      "stockholders_equity": "N/A",
      "book_value_per_share": "N/A"
    }
  },
  "error": null,
  "tool": "

### 2.2 公司基本資訊

In [19]:
# 測試：取得公司基本資訊
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_company_profile",
        {"symbol": "2330"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

{
  "success": true,
  "data": {
    "出表日期": "1141018",
    "公司代號": "2330",
    "公司名稱": "台灣積體電路製造股份有限公司",
    "公司簡稱": "台積電",
    "外國企業註冊地國": "－ ",
    "產業別": "24",
    "住址": "新竹科學園區力行六路8號",
    "營利事業統一編號": "22099131",
    "董事長": "魏哲家",
    "總經理": "總裁: 魏哲家",
    "發言人": "黃仁昭",
    "發言人職稱": "資深副總經理暨財務長",
    "代理發言人": "高孟華",
    "總機電話": "03-5636688",
    "成立日期": "19870221",
    "上市日期": "19940905",
    "普通股每股面額": "新台幣                 10.0000元",
    "實收資本額": "259326155210",
    "私募股數": "0",
    "特別股": "0",
    "編制財務報表類型": "1",
    "股票過戶機構": "中國信託商業銀行 代理部",
    "過戶電話": "02-6636-5566",
    "過戶地址": "台北市重慶南路一段83號5樓",
    "簽證會計師事務所": "勤業眾信聯合會計師事務所",
    "簽證會計師1": "吳世宗",
    "簽證會計師2": "林尚志",
    "英文簡稱": "TSMC",
    "英文通訊地址": "No. 8, Li-Hsin Rd. 6, Hsinchu Science Park,Hsin-Chu 300096, Taiwan, R.O.C.",
    "傳真機號碼": "03-5797337",
    "電子郵件信箱": "invest@tsmc.com",
    "網址": "https://www.tsmc.com",
    "已發行普通股數或TDR原股發行股數": "25932615521"
  },
  "error": null,
  "tool": "company_profile",
  "timestamp": 

### 2.3 股利資訊

In [20]:
# 測試：取得股利分配資訊
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_company_dividend",
        {"symbol": "2330"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

{
  "success": true,
  "data": {
    "出表日期": "1141018",
    "公司代號": "2330",
    "公司名稱": "台積電",
    "決議（擬議）進度": "董事會決議",
    "股利年度": "114",
    "股利所屬年(季)度": "第2季",
    "股利所屬期間": "1140401~1140630",
    "期別": "1",
    "董事會（擬議）股利分派日": "1140812",
    "股東會日期": "",
    "期初未分配盈餘/待彌補虧損(元)": "3721467292144",
    "本期淨利(淨損)(元)": "398273102398",
    "可分配盈餘(元)": "3938185547000",
    "分配後期末未分配盈餘(元)": "3808522634395",
    "股東配發-盈餘分配之現金股利(元/股)": "5.00000000",
    "股東配發-法定盈餘公積發放之現金(元/股)": "0.0",
    "股東配發-資本公積發放之現金(元/股)": "0.0",
    "股東配發-股東配發之現金(股利)總金額(元)": "129662912605",
    "股東配發-盈餘轉增資配股(元/股)": "0.0",
    "股東配發-法定盈餘公積轉增資配股(元/股)": "0.0",
    "股東配發-資本公積轉增資配股(元/股)": "0.0",
    "股東配發-股東配股總股數(股)": "0",
    "摘錄公司章程-股利分派部分": "",
    "備註": "無。"
  },
  "error": null,
  "tool": "dividend",
  "timestamp": "2025-10-20T00:05:02.003645",
  "metadata": {
    "company_code": "2330",
    "source": "TWSE OpenAPI"
  }
}


In [21]:
# 測試：取得除權息行事曆 (單一公司)
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_dividend_rights_schedule",
        {"symbol": "2330"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

{
  "success": false,
  "data": null,
  "error": "No dividend schedule found for 2330",
  "tool": "dividend_schedule",
  "timestamp": "2025-10-20T00:05:13.050731",
  "metadata": {
    "symbol": "2330"
  }
}


In [22]:
# 測試：取得除權息行事曆 (所有公司)
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_dividend_rights_schedule",
        {"symbol": ""}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

{
  "success": true,
  "data": {
    "dividend_schedule": [
      {
        "Date": "1141023",
        "Code": "0056",
        "Name": "元大高股息",
        "Exdividend": "息",
        "StockDividendRatio": "",
        "SubscriptionRatio": "",
        "SubscriptionPricePerShare": "",
        "CashDividend": "",
        "SharesOffered": "",
        "SharesEmpOwner": "",
        "SharesholderOwner": "",
        "StockHoldingRatio": ""
      },
      {
        "Date": "1141023",
        "Code": "006204",
        "Name": "永豐臺灣加權",
        "Exdividend": "息",
        "StockDividendRatio": "",
        "SubscriptionRatio": "",
        "SubscriptionPricePerShare": "",
        "CashDividend": "",
        "SharesOffered": "",
        "SharesEmpOwner": "",
        "SharesholderOwner": "",
        "StockHoldingRatio": ""
      },
      {
        "Date": "1141020",
        "Code": "00714",
        "Name": "群益道瓊美國地產",
        "Exdividend": "息",
        "StockDividendRatio": "",
        "SubscriptionRatio":

### 2.4 營收與估值

In [23]:
# 測試：取得月營收
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_company_monthly_revenue",
        {"symbol": "2330"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

{
  "success": true,
  "data": {
    "出表日期": "1141017",
    "資料年月": "11409",
    "公司代號": "2330",
    "公司名稱": "台積電",
    "產業別": "半導體業",
    "營業收入-當月營收": "330980920",
    "營業收入-上月營收": "335771691",
    "營業收入-去年當月營收": "251872717",
    "營業收入-上月比較增減(%)": "-1.426794196298103",
    "營業收入-去年同月增減(%)": "31.40800795824186",
    "累計營業收入-當月累計營收": "2762963851",
    "累計營業收入-去年累計營收": "2025846521",
    "累計營業收入-前期比較增減(%)": "36.385645326978846",
    "備註": "-"
  },
  "error": null,
  "tool": "revenue",
  "timestamp": "2025-10-20T00:06:09.330569",
  "metadata": {
    "company_code": "2330",
    "source": "TWSE OpenAPI"
  }
}


In [24]:
# 測試：取得估值比率
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_stock_valuation_ratios",
        {"symbol": "2330"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

{
  "success": true,
  "data": {
    "Date": "1141017",
    "Code": "2330",
    "Name": "台積電",
    "PEratio": "25.76",
    "DividendYield": "1.17",
    "PBratio": "8.21"
  },
  "error": null,
  "tool": "valuation",
  "timestamp": "2025-10-20T00:06:17.749243",
  "metadata": {
    "company_code": "2330",
    "source": "TWSE Exchange Report"
  }
}


## 3. 交易統計工具測試

### 3.1 月交易統計

In [25]:
# 測試：取得月交易資訊
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_stock_monthly_trading",
        {"symbol": "2330"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

{
  "success": true,
  "data": {
    "Month": "11409",
    "Code": "2330",
    "Name": "台積電",
    "HighestPrice": "1355.00",
    "LowestPrice": "1145.00",
    "WeightedAvgPriceAB": "1255.99",
    "Transaction": "1066189",
    "TradeValueA": "848522727420",
    "TradeVolumeB": "675579256",
    "TurnoverRatio": "2.60"
  },
  "error": null,
  "tool": "trading_statistics",
  "timestamp": "2025-10-20T00:06:25.805331",
  "metadata": {
    "source": "TWSE Exchange Report",
    "company_code": "2330"
  }
}


In [26]:
# 測試：取得月平均價格
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_stock_monthly_average",
        {"symbol": "2330"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

{
  "success": true,
  "data": {
    "Month": "11409",
    "Code": "2330",
    "Name": "台積電",
    "HighestPrice": "1355.00",
    "LowestPrice": "1145.00",
    "WeightedAvgPriceAB": "1255.99",
    "Transaction": "1066189",
    "TradeValueA": "848522727420",
    "TradeVolumeB": "675579256",
    "TurnoverRatio": "2.60"
  },
  "error": null,
  "tool": "trading_statistics",
  "timestamp": "2025-10-20T00:06:34.998891",
  "metadata": {
    "source": "TWSE Exchange Report",
    "company_code": "2330"
  }
}


### 3.2 年交易統計

In [27]:
# 測試：取得年交易資訊
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_stock_yearly_trading",
        {"symbol": "2330"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

{
  "success": true,
  "data": {
    "Year": "113",
    "Code": "2330",
    "Name": "台積電",
    "TradeVolume": "10717183197",
    "TradeValue": "9504674734734",
    "Transaction": "16643941",
    "HighestPrice": "1100.00",
    "HDate": "10月18日",
    "LowestPrice": "574.00",
    "LDate": "1月05日",
    "AvgClosingPrice": "889.07"
  },
  "error": null,
  "tool": "trading_statistics",
  "timestamp": "2025-10-20T00:06:43.245562",
  "metadata": {
    "source": "TWSE Exchange Report",
    "company_code": "2330"
  }
}


## 4. 市場資訊工具測試

### 4.1 融資融券

In [28]:
# 測試：取得融資融券資訊
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_margin_trading_info",
        {}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

{
  "success": true,
  "data": {
    "margin_data": [
      {
        "股票代號": "0050",
        "股票名稱": "元大台灣50",
        "融資買進": "1002",
        "融資賣出": "557",
        "融資現金償還": "32",
        "融資前日餘額": "7472",
        "融資今日餘額": "7885",
        "融資限額": "3313125",
        "融券買進": "40",
        "融券賣出": "108",
        "融券現券償還": "",
        "融券前日餘額": "1656",
        "融券今日餘額": "1724",
        "融券限額": "3313125",
        "資券互抵": "14",
        "註記": " "
      },
      {
        "股票代號": "0051",
        "股票名稱": "元大中型100",
        "融資買進": "",
        "融資賣出": "",
        "融資現金償還": "",
        "融資前日餘額": "1",
        "融資今日餘額": "1",
        "融資限額": "5750",
        "融券買進": "",
        "融券賣出": "",
        "融券現券償還": "",
        "融券前日餘額": "",
        "融券今日餘額": "",
        "融券限額": "5750",
        "資券互抵": "",
        "註記": " "
      },
      {
        "股票代號": "0052",
        "股票名稱": "富邦科技",
        "融資買進": "109",
        "融資賣出": "16",
        "融資現金償還": "",
        "融資前日餘額": "334",
        "融資今日餘額": "427",
  

### 4.2 即時交易統計

In [29]:
# 測試：取得即時交易統計
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_real_time_trading_stats",
        {}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

{
  "success": true,
  "data": {
    "trading_stats": [
      {
        "Time": "132915",
        "AccBidOrders": "17446656",
        "AccBidVolume": "22651860",
        "AccAskOrders": "17839817",
        "AccAskVolume": "12491349",
        "AccTransaction": "2348053",
        "AccTradeVolume": "8566995",
        "AccTradeValue": "499358"
      },
      {
        "Time": "132920",
        "AccBidOrders": "17455229",
        "AccBidVolume": "22667426",
        "AccAskOrders": "17844893",
        "AccAskVolume": "12494025",
        "AccTransaction": "2348053",
        "AccTradeVolume": "8566995",
        "AccTradeValue": "499358"
      },
      {
        "Time": "132925",
        "AccBidOrders": "17462440",
        "AccBidVolume": "22698396",
        "AccAskOrders": "17849018",
        "AccAskVolume": "12525685",
        "AccTransaction": "2348053",
        "AccTradeVolume": "8566995",
        "AccTradeValue": "499358"
      },
      {
        "Time": "132930",
        "AccBidOrders": "

### 4.3 ETF排名

In [30]:
# 測試：取得ETF定期定額排名
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_etf_regular_investment_ranking",
        {}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

{
  "success": true,
  "data": {
    "ranking_date": null,
    "rankings": [
      {
        "No": "1",
        "STOCKsSecurityCode": "2330",
        "STOCKsName": "台積電",
        "STOCKsNumberofTradingAccounts": "118276",
        "ETFsSecurityCode": "0050",
        "ETFsName": "元大台灣50",
        "ETFsNumberofTradingAccounts": "535855"
      },
      {
        "No": "2",
        "STOCKsSecurityCode": "2886",
        "STOCKsName": "兆豐金",
        "STOCKsNumberofTradingAccounts": "29805",
        "ETFsSecurityCode": "00878",
        "ETFsName": "國泰永續高股息",
        "ETFsNumberofTradingAccounts": "294924"
      },
      {
        "No": "3",
        "STOCKsSecurityCode": "2884",
        "STOCKsName": "玉山金",
        "STOCKsNumberofTradingAccounts": "29751",
        "ETFsSecurityCode": "0056",
        "ETFsName": "元大高股息",
        "ETFsNumberofTradingAccounts": "272193"
      },
      {
        "No": "4",
        "STOCKsSecurityCode": "2317",
        "STOCKsName": "鴻海",
        "STOCKsNumberofTrad

### 4.4 市場指數

In [35]:
# 測試：取得發行量加權股價指數
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_market_index_info",
        {}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

{
  "success": true,
  "data": {
    "日期": "1141017",
    "指數": "發行量加權股價指數",
    "收盤指數": "27302",
    "漲跌": "-",
    "漲跌點數": "345.50",
    "漲跌百分比": "-1.25",
    "特殊處理註記": ""
  },
  "error": null,
  "tool": "index_info",
  "timestamp": "2025-10-20T00:22:16.866177",
  "metadata": {
    "source": "TWSE Market Index Report"
  }
}


### 4.5 歷史指數

In [37]:
# 測試：取得歷史指數資料
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_market_historical_index",
        {}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

{
  "success": true,
  "data": {
    "indices": [
      {
        "日期": "1141017",
        "指數": "發行量加權股價指數",
        "收盤指數": "27302",
        "漲跌": "-",
        "漲跌點數": "345.50",
        "漲跌百分比": "-1.25",
        "特殊處理註記": ""
      },
      {
        "日期": "1141017",
        "指數": "未含金融指數",
        "收盤指數": "24144",
        "漲跌": "-",
        "漲跌點數": "327.43",
        "漲跌百分比": "-1.34",
        "特殊處理註記": ""
      },
      {
        "日期": "1141017",
        "指數": "未含電子指數",
        "收盤指數": "18906",
        "漲跌": "-",
        "漲跌點數": "26.65",
        "漲跌百分比": "-0.14",
        "特殊處理註記": ""
      },
      {
        "日期": "1141017",
        "指數": "臺灣50指數",
        "收盤指數": "24457",
        "漲跌": "-",
        "漲跌點數": "395.88",
        "漲跌百分比": "-1.59",
        "特殊處理註記": ""
      },
      {
        "日期": "1141017",
        "指數": "臺灣中型100指數",
        "收盤指數": "21498",
        "漲跌": "-",
        "漲跌點數": "142.09",
        "漲跌百分比": "-0.66",
        "特殊處理註記": ""
      },
      {
        "日期": "1141017

## 5. 外資工具測試

### 5.1 外資持股（按產業）

In [48]:
# 測試：取得外資各產業持股
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_foreign_investment_by_industry",
        {"count": 8}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

{
  "success": true,
  "data": {
    "industry_foreign_investment": [
      {
        "IndustryCat": "ETF",
        "Numbers": "203",
        "ShareNumber": "163170802913",
        "ForeignMainlandAreaShare": "2347329612",
        "Percentage": "1.44"
      },
      {
        "IndustryCat": "水泥工業",
        "Numbers": "8",
        "ShareNumber": "14092571612",
        "ForeignMainlandAreaShare": "2360076435",
        "Percentage": "16.75"
      },
      {
        "IndustryCat": "食品工業",
        "Numbers": "24",
        "ShareNumber": "14012118644",
        "ForeignMainlandAreaShare": "2720621007",
        "Percentage": "19.42"
      },
      {
        "IndustryCat": "塑膠工業",
        "Numbers": "22",
        "ShareNumber": "31845231713",
        "ForeignMainlandAreaShare": "6938995241",
        "Percentage": "21.79"
      },
      {
        "IndustryCat": "紡織纖維",
        "Numbers": "42",
        "ShareNumber": "18985472568",
        "ForeignMainlandAreaShare": "1887801723",
        "Percen

### 5.2 外資持股前20名

In [39]:
# 測試：取得外資持股前20名
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_top_foreign_holdings",
        {}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

{
  "success": true,
  "data": {
    "top_foreign_holdings": [
      {
        "Rank": "1",
        "Code": "2923",
        "Name": "鼎固-KY",
        "ShareNumber": "1740296982",
        "AvailableShare": "10988659",
        "SharesHeld": "1729308323",
        "AvailableInvestPer": "0.63",
        "SharesHeldPer": "99.36",
        "Upperlimit": "100.00"
      },
      {
        "Rank": "2",
        "Code": "8482",
        "Name": "商億-KY",
        "ShareNumber": "107444000",
        "AvailableShare": "8811984",
        "SharesHeld": "98632016",
        "AvailableInvestPer": "8.20",
        "SharesHeldPer": "91.79",
        "Upperlimit": "100.00"
      },
      {
        "Rank": "3",
        "Code": "6890",
        "Name": "來億-KY",
        "ShareNumber": "249400000",
        "AvailableShare": "29886886",
        "SharesHeld": "219513114",
        "AvailableInvestPer": "11.98",
        "SharesHeldPer": "88.01",
        "Upperlimit": "100.00"
      },
      {
        "Rank": "4",
        "C

## 6. 節假日工具測試

### 6.1 節假日資訊

In [40]:
# 測試：查詢元旦
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_taiwan_holiday_info",
        {"date": "2025-01-01"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

{
  "success": true,
  "data": {
    "date": "20250101",
    "name": "中華民國開國紀念日",
    "is_holiday": true,
    "holiday_category": "放假之紀念日及節日",
    "description": "全國各機關學校放假一日。"
  },
  "error": null,
  "tool": "holiday_tool",
  "timestamp": "2025-10-20T00:32:16.476483",
  "metadata": {}
}


In [41]:
# 測試：查詢中秋節
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_taiwan_holiday_info",
        {"date": "2025-10-06"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

{
  "success": true,
  "data": {
    "date": "20251006",
    "name": "中秋節",
    "is_holiday": true,
    "holiday_category": "放假之紀念日及節日",
    "description": "全國各機關學校放假一日。"
  },
  "error": null,
  "tool": "holiday_tool",
  "timestamp": "2025-10-20T00:32:24.101189",
  "metadata": {}
}


In [42]:
# 測試：查詢一般工作日
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "get_taiwan_holiday_info",
        {"date": "2025-10-07"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

{
  "success": true,
  "data": {
    "date": "20251007",
    "name": "",
    "is_holiday": false,
    "holiday_category": "",
    "description": ""
  },
  "error": null,
  "tool": "holiday_tool",
  "timestamp": "2025-10-20T00:32:29.313297",
  "metadata": {}
}


### 6.2 交易日檢查

In [43]:
# 測試：檢查元旦是否為交易日
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "check_taiwan_trading_day",
        {"date": "2025-01-01"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

{
  "success": true,
  "data": {
    "date": "2025-01-01",
    "is_trading_day": false,
    "is_weekend": false,
    "is_holiday": true,
    "holiday_name": "中華民國開國紀念日",
    "reason": "國定假日（中華民國開國紀念日）"
  },
  "error": null,
  "tool": "trading_day_tool",
  "timestamp": "2025-10-20T00:32:36.686202",
  "metadata": {}
}


In [44]:
# 測試：檢查週六是否為交易日
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "check_taiwan_trading_day",
        {"date": "2025-10-11"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

{
  "success": true,
  "data": {
    "date": "2025-10-11",
    "is_trading_day": false,
    "is_weekend": true,
    "is_holiday": true,
    "holiday_name": "",
    "reason": "週末且為國定假日（）"
  },
  "error": null,
  "tool": "trading_day_tool",
  "timestamp": "2025-10-20T00:32:40.842312",
  "metadata": {}
}


In [45]:
# 測試：檢查一般工作日是否為交易日
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    result = await server.session.call_tool(
        "check_taiwan_trading_day",
        {"date": "2025-10-07"}
    )
    print(json.dumps(json.loads(result.content[0].text), ensure_ascii=False, indent=2))

{
  "success": true,
  "data": {
    "date": "2025-10-07",
    "is_trading_day": true,
    "is_weekend": false,
    "is_holiday": false,
    "holiday_name": "",
    "reason": "是交易日"
  },
  "error": null,
  "tool": "trading_day_tool",
  "timestamp": "2025-10-20T00:32:46.121204",
  "metadata": {}
}
